In [29]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from IPython.display import Image
import torch

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
sns.set_style('darkgrid')

In [31]:
DATA_DIR = '../data/UTKFace'

# Prepare samples for unbiased FID statistic calculation

## Single-attribute

combine all data across splits to maximize number of samples

In [1]:
splits = ['test', 'val', 'train']

In [2]:
data = []
labels = []
for split in splits:
    d = torch.load(os.path.join(DATA_DIR, '{}_UTK_64x64.pt'.format(split)))
    l = torch.load(os.path.join(DATA_DIR, '{}_labels_UTK_64x64.pt'.format(split)))
    data.append(d)
    labels.append(l)

NameError: name 'torch' is not defined

In [34]:
data = torch.cat(data)
labels = torch.cat(labels)

In [35]:
data.shape, labels.shape

(torch.Size([23705, 3, 64, 64]), torch.Size([23705]))

In [36]:
# get minimum value of class
val, freq = np.unique(labels.data.numpy(), return_counts=True)
min_value = min(freq)
print(val, freq)
print(min(freq))

[0. 1.] [10078 13627]
10078


In [37]:
samples = []
ys = []
for i in range(len(val)):
    idx = np.where(labels.data.numpy() == i)[0][0:min_value]
    samples.append(data[idx])
    ys.append(labels[idx])

In [38]:
len(samples)

2

In [39]:
samples[0].shape, ys[0].shape, samples[1].shape, ys[1].shape

(torch.Size([10078, 3, 64, 64]),
 torch.Size([10078]),
 torch.Size([10078, 3, 64, 64]),
 torch.Size([10078]))

In [40]:
samples = torch.cat(samples)
samples = samples.numpy()
samples.shape

(20156, 3, 64, 64)

In [41]:
# now we should convert to numpy
samples = np.array(samples)

In [42]:
samples.shape

(20156, 3, 64, 64)

In [43]:
np.savez('../fid_stats/UTKFace/unbiased_all_race_samples.npz', **{'x':samples})